In [1]:
import numpy as np
import random
import itertools
from operator import attrgetter
from collections import defaultdict
import matplotlib.pyplot as plt

In [10]:
suits = {'Hearts', 'Clubs', 'Spades', 'Diamonds'}

card_name_map = {
            'Two': 2,
            'Three': 3,
            'Four': 4,
            'Five': 5,
            'Six': 6,
            'Seven': 7,
            'Eight': 8,
            'Nine': 9,
            'Ten': 10,
            'Jack': 11, 
            'Queen': 12, 
            'King': 13, 
            'Ace': 14
            }

class Card:
    def __init__(self, name, suit):
        self.suit = suit
        self.name = name
        if self.name == 'Six':
            self.name_plural = 'Sixes'
        else:
            self.name_plural = self.name + 's'
        self.value = card_name_map[self.name]
    
    def __str__(self):
        return '{} of {}'.format(self.name, self.suit)
    
    def __eq__(self, other):
        return self.value == other.value

In [30]:
class Deck:
    def __init__(self):
        self.deck = []
        self.card_location_map = {}
        for suit in suits:
            for card_name in card_name_map:
                self.deck.append(Card(card_name, suit))
        self.shuffle()
                
    def __str__(self):
        return ''.join(['{} \n'.format(card) for card in self.deck])
    
    def assign_locations(self):
        self.card_location_map = {}
        for i, card in enumerate(self.deck):
            self.card_location_map[str(card)] = i
        
    
    def shuffle(self):
        random.shuffle(self.deck)
        self.assign_locations()
    
    def remove(self, card):
        card = self.deck.pop(self.card_location_map[str(card)])
        self.assign_locations()
        return card
        
    
    def draw(self):
        card = self.deck.pop(0)
        self.assign_locations()
        return card
    
    def __str__(self):
        return ''.join(['{} \n'.format(card) for card in self.deck])

In [34]:
class Hand:
    def __init__(self, cards):
        self.cards = cards

In [49]:
class Player:
    def __init__(self, name):
        self.name = name    
        self.cards = []
        self.hand = None
        self.possible_hands = []
        self.best_hand = None
        
    def __str__(self):
        return '{}{}'.format('Player: {}\n'.format(self.name), ''.join(['{} \n'.format(card) for card in self.cards]))
                                                                                               

In [50]:
class Table:
    def __init__(self, deck):
        self.deck = deck
        self.board_cards = []
        self.burned_cards = []

In [64]:
class Game:
    def __init__(self, players):
        self.players = [Player('{}'.format(i)) for i in range(players)]
        self.deck = Deck()
        self.table = Table(self.deck)
    
    def deal(self):
        for i in range(2):
            for player in self.players:
                player.cards.append(self.deck.draw())
    
    def burn(self):
        self.table.burned_cards.append(self.deck.draw())
        
    def draw(self):
        self.table.board_cards.append(self.deck.draw())
                
    def flop(self):
        self.burn()
        for i in range(3):
            self.draw()
    
    def turn(self):
        self.burn()
        self.draw()
    
    def river(self):
        self.burn()
        self.draw()
        
    def get_possible_hands(self, player):
        cards = player.cards + self.table.board_cards
        hands = []
        for comb in itertools.combinations(cards, 5):
            hands.append(list(comb))
        player.possible_hands = hands
    
    def __str__(self):
        return ''.join(['{}\n'.format(player) for player in self.players])
    

In [65]:
g = Game(4)
g.deal()
g.flop()
g.turn()
g.river()

print(g)
print([str(card) for card in g.table.board_cards])

Player: 0
Eight of Clubs 
Four of Diamonds 

Player: 1
Four of Hearts 
Ace of Hearts 

Player: 2
King of Hearts 
Five of Spades 

Player: 3
King of Spades 
Two of Spades 


['Queen of Diamonds', 'Three of Hearts', 'Queen of Clubs', 'Ace of Clubs', 'Ten of Clubs']


In [66]:
for player in g.players:
    g.get_possible_hands(player)

In [68]:
for player in g.players:
    print([str(card) for card in player.possible_hands])

['[<__main__.Card object at 0x7fc8f19d00d0>, <__main__.Card object at 0x7fc8f19eea90>, <__main__.Card object at 0x7fc8f1ac2190>, <__main__.Card object at 0x7fc8f1abd0d0>, <__main__.Card object at 0x7fc8f19d0cd0>]', '[<__main__.Card object at 0x7fc8f19d00d0>, <__main__.Card object at 0x7fc8f19eea90>, <__main__.Card object at 0x7fc8f1ac2190>, <__main__.Card object at 0x7fc8f1abd0d0>, <__main__.Card object at 0x7fc8f19d0b50>]', '[<__main__.Card object at 0x7fc8f19d00d0>, <__main__.Card object at 0x7fc8f19eea90>, <__main__.Card object at 0x7fc8f1ac2190>, <__main__.Card object at 0x7fc8f1abd0d0>, <__main__.Card object at 0x7fc8f19d0990>]', '[<__main__.Card object at 0x7fc8f19d00d0>, <__main__.Card object at 0x7fc8f19eea90>, <__main__.Card object at 0x7fc8f1ac2190>, <__main__.Card object at 0x7fc8f19d0cd0>, <__main__.Card object at 0x7fc8f19d0b50>]', '[<__main__.Card object at 0x7fc8f19d00d0>, <__main__.Card object at 0x7fc8f19eea90>, <__main__.Card object at 0x7fc8f1ac2190>, <__main__.Card 